# Set global parameters

In [47]:
use_accelerate = True
num_devices = 2 if use_accelerate else 1

GENERATION = "11b"
SWEEP_ID = 1
base_dir = '/home/lange/fantastic-umbrella/runs'

boolean_args = {
    'with_tracking' : [True],
}

# Set run(s) parameters

In [48]:
value_args ={
    'task' : ['rte'],
    'model_name_or_path' : ['bert-base-cased'],
    'per_device_train_batch_size' : [32 // num_devices],
    'per_device_eval_batch_size' : [16 // num_devices],
    'learning_rate': [1e-6],
    'weight_decay' : [0],
    'num_train_epochs' : [100000],
    'max_train_steps' : [100000],
    'lr_scheduler_type' : ['linear'],
    'num_warmup_steps' : [500],
    'seed' : [0],
    'report_to' : ['all'],
    'insert_dropout' : [0.1],
    # 'catch_dropout' : [0],
    'training_size' : [32],
    'beta1' : [0.9],
    'beta2' : [0.999],
    'early_stopping_patience' : [5000],
    'early_stopping_min_delta' : [0],
    'original_gradient_fraction' : [0]
}


# Util functions

In [49]:
def compose_instruction(value_args, boolean_args, use_accelerate=False):
    if use_accelerate:
        instruction = 'accelerate launch run_glue_no_trainer_modded.py'
    else:
        instruction = 'python run_glue_no_trainer_modded.py'

    for k, v in value_args.items():
        instruction += f' --{k} {v}'

    for k, v in boolean_args.items():
        if v:
            instruction += f' --{k}'

    return instruction

In [50]:
def determine_run_type(list_of_value_args):
    if 'catch_dropout' in list_of_value_args[0]:
        return "modded"
    else:
        return "vanilla"

# Create combinations

In [51]:
from itertools import product
keys, values = zip(*value_args.items())
list_of_value_args = [dict(zip(keys, v)) for v in product(*values)]
print(len(list_of_value_args))
list_of_value_args[0]

1


{'task': 'rte',
 'model_name_or_path': 'bert-base-cased',
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 8,
 'learning_rate': 1e-06,
 'weight_decay': 0,
 'num_train_epochs': 100000,
 'max_train_steps': 100000,
 'lr_scheduler_type': 'linear',
 'num_warmup_steps': 500,
 'seed': 0,
 'report_to': 'all',
 'insert_dropout': 0.1,
 'training_size': 32,
 'beta1': 0.9,
 'beta2': 0.999,
 'early_stopping_patience': 5000,
 'early_stopping_min_delta': 0,
 'original_gradient_fraction': 0}

# Generate instructions

In [52]:
from math import log, ceil
sweep_type = determine_run_type(list_of_value_args)
print(f'Detected sweep type: {sweep_type}')
magnitude = ceil(log(len(list_of_value_args),10))
print(f'Number of experiments in this run in the range of 10^{magnitude}')
sweep_identifier = f'{GENERATION}_{str(SWEEP_ID).zfill(2)}'

with open(f'instructions_{sweep_identifier}.txt','w', newline='\n') as f:
    sweep_identifier += f'_{sweep_type}'
    for idx, value_args in enumerate(list_of_value_args):
        run_id = str(idx).zfill(magnitude)
        output_dir = f'{base_dir}/sweep_{sweep_identifier}/run_{run_id}'
        value_args["output_dir"] = output_dir

        instruction = compose_instruction(
            value_args = value_args, 
            boolean_args = boolean_args,
            use_accelerate = use_accelerate
        )

        f.write(instruction + '\n')


Detected sweep type: vanilla
Number of experiments in this run in the range of 10^0


In [53]:
# run_id = 0
# run_type = 'single'


# output_dir = f'{base_dir}/{run_name}'

# value_args ={
#     'task' : 'rte',
#     'model_name_or_path' : 'bert-base-cased',
#     'per_device_train_batch_size' : 32 // num_devices,
#     'per_device_eval_batch_size' : 16 // num_devices,
#     'learning_rate': 1e-6,
#     'weight_decay' : 0,
#     'num_train_epochs' : 100000,
#     'max_train_steps' : 100000,
#     'lr_scheduler_type' : 'linear',
#     'num_warmup_steps' : 500,
#     'output_dir' : output_dir,
#     'seed' : 0,
#     'report_to' : 'all',
#     'insert_dropout' : 0.1,
#     'catch_dropout' : 0,
#     'training_size' : 32,
#     'beta1' : 0.9,
#     'beta2' : 0.999,
#     'early_stopping_patience' : 5000,
#     'early_stopping_min_delta' : 0,
#     'original_gradient_fraction' : 0
# }
# boolean_args = {
#     'with_tracking' : True,

# }

# # Compose instruction
# instruction = compose_instruction(value_args = value_args, 
#                                   boolean_args = boolean_args,
#                                   use_accelerate = use_accelerate)

# print(instruction)
# # Write instruction
# with open(f'instruction_{GENERATION}_{run_id}.txt', 'w') as f:
#     f.write(instruction)